In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import backtrader as bt # Import the backtrader platform
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import seaborn as sns
from collections import deque

# BackTrader

In [188]:
def run_backtest(aStrategy,year_start=2009,year_end=2018,label="s"):
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(aStrategy)

    datapath = os.path.join('../../../../datas/spx-1950-2018.csv')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(year_start, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(year_end, 1, 1),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='%s.csv'%label,csv=True)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

## Momentum Strategy
Momentum(period_BB,num_std):        
* Long if asset crosses over Upper Band         
-close if asset crosses under Middle Band       
* Short if asset crosses down Lower Band        
-close if asset crosses from below Middle Band 

In [81]:
class MomentumStrategy(bt.Strategy):
    '''
    This strategy is loosely based on some of the examples from the Van
    K. Tharp book: *Trade Your Way To Financial Freedom*. The logic:

      - Enter the market if:
        - The MACD.macd line crosses the MACD.signal line to the upside
        - The Simple Moving Average has a negative direction in the last x
          periods (actual value below value x periods ago)

     - Set a stop price x times the ATR value away from the close

     - If in the market:

       - Check if the current close has gone below the stop price. If yes,
         exit.
       - If not, update the stop price if the new stop price would be higher
         than the current
    '''

    params = (
        # Standard MACD Parameters
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 1),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
    )
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def __init__(self):

        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # MACD Indicator
        self.macd = bt.indicators.MACD(self.datas[0],
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig)

        # Cross of macd.macd and macd.signal
        self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
        self.mcross2 = bt.indicators.CrossDown(self.macd.macd, self.macd.signal)
        # To set the stop price
        self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)

        # Control market trend
        self.sma = bt.indicators.SMA(self.data, period=self.p.smaperiod)
        self.smadir = self.sma - self.sma(-self.p.dirperiod)

    def start(self):
        self.order = None  # sentinel to avoid operrations on pending order

    def next(self):
        # Always long or short $10000 securities
        size = 10000./self.dataclose[0]
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f ; Position %f' % (self.dataclose[0],self.position.size))
        
        if self.order:
            return  # pending order execution

        if not self.position:  # not in the market
            if self.mcross[0] > 0.0 and self.smadir < 0.0 and self.macd.macd[0]>=self.macd.macd[-1]:
                self.order = self.buy(size=size)
                self.log('{mo}BUY CREATE, %.2f' % self.dataclose[0])
                pdist = self.atr[0] * self.p.atrdist
                self.pstop = self.data.close[0] - pdist
            
            elif self.mcross[0] < 0.0 and self.smadir > 0.0 and self.macd.macd[0]<=self.macd.macd[-1]:
                self.order = self.sell(size=size)
                self.log('{mo}SELL CREATE, %.2f' % self.dataclose[0])
                pdist = self.atr[0] * self.p.atrdist
                self.pstop = self.data.close[0] + pdist
            

        else:  # in the market
            if self.position.size>0:
                pclose = self.data.close[0]
                pstop = self.pstop
                pdist = self.atr[0] * self.p.atrdist
                
                # Reverse long position
                if self.mcross[0] < 0.0 and self.smadir > 0.0 and self.macd.macd[0]<=self.macd.macd[-1]:
                    self.close()
                    self.order = self.sell(size=size)
                    self.log('{mo}[Reverse]SELL CREATE, %.2f' % self.dataclose[0])
                    self.pstop = self.data.close[0] + pdist

                # Close long position
                elif pclose < pstop:
                    self.close()  # stop met - get out
                    self.log('{mo}[Exit]SELL CREATE, %.2f' % self.dataclose[0])
                else:
                    # Update only if greater than
                    self.pstop = max(pstop, pclose - pdist)
            
            elif self.position.size<0:
                pclose = self.data.close[0]
                pstop = self.pstop
                pdist = self.atr[0] * self.p.atrdist
                
                # Reverse short position
                if self.mcross[0] > 0.0 and self.smadir < 0.0 and self.macd.macd[0]>=self.macd.macd[-1]:
                    self.order = self.buy(size=size)
                    self.log('{mo}[Reverse]BUY CREATE, %.2f' % self.dataclose[0])
                    pdist = self.atr[0] * self.p.atrdist
                    self.pstop = self.data.close[0] - pdist
                    
                # Close short position
                if pclose > pstop:
                    self.close()  # stop met - get out
                    self.log('{mo}[Exit]BUY CREATE, %.2f' % self.dataclose[0])
                else:
                    # Update only if greater than
                    self.pstop = min(pstop, pclose + pdist)
run_backtest(MomentumStrategy,'mo')            

Starting Portfolio Value: 100000.00
2009-02-20, Close, 770.05 ; Position 0.000000
2009-02-23, Close, 743.33 ; Position 0.000000
2009-02-24, Close, 773.14 ; Position 0.000000
2009-02-25, Close, 764.90 ; Position 0.000000
2009-02-26, Close, 752.83 ; Position 0.000000
2009-02-27, Close, 735.09 ; Position 0.000000
2009-03-02, Close, 700.82 ; Position 0.000000
2009-03-03, Close, 696.33 ; Position 0.000000
2009-03-04, Close, 712.87 ; Position 0.000000
2009-03-05, Close, 682.55 ; Position 0.000000
2009-03-06, Close, 683.38 ; Position 0.000000
2009-03-09, Close, 676.53 ; Position 0.000000
2009-03-10, Close, 719.60 ; Position 0.000000
2009-03-11, Close, 721.36 ; Position 0.000000
2009-03-12, Close, 750.74 ; Position 0.000000
2009-03-12, {mo}BUY CREATE, 750.74
2009-03-13, BUY EXECUTED, 751.97
2009-03-13, Close, 756.55 ; Position 13.320191
2009-03-16, Close, 753.89 ; Position 13.320191
2009-03-17, Close, 778.12 ; Position 13.320191
2009-03-18, Close, 794.35 ; Position 13.320191
2009-03-19, Close,

2010-10-21, Close, 1180.26 ; Position -8.577065
2010-10-22, Close, 1183.08 ; Position -8.577065
2010-10-22, {mo}[Exit]BUY CREATE, 1183.08
2010-10-25, BUY EXECUTED, 1184.74
2010-10-25, Close, 1185.62 ; Position 0.000000
2010-10-26, Close, 1185.64 ; Position 0.000000
2010-10-27, Close, 1182.45 ; Position 0.000000
2010-10-28, Close, 1183.78 ; Position 0.000000
2010-10-29, Close, 1183.26 ; Position 0.000000
2010-11-01, Close, 1184.38 ; Position 0.000000
2010-11-02, Close, 1193.57 ; Position 0.000000
2010-11-03, Close, 1197.96 ; Position 0.000000
2010-11-04, Close, 1221.06 ; Position 0.000000
2010-11-05, Close, 1225.85 ; Position 0.000000
2010-11-08, Close, 1223.25 ; Position 0.000000
2010-11-09, Close, 1213.40 ; Position 0.000000
2010-11-10, Close, 1218.71 ; Position 0.000000
2010-11-11, Close, 1213.54 ; Position 0.000000
2010-11-12, Close, 1199.21 ; Position 0.000000
2010-11-12, {mo}SELL CREATE, 1199.21
2010-11-15, SELL EXECUTED, 1200.44
2010-11-15, Close, 1197.75 ; Position -8.338823
201

2012-06-21, {mo}[Exit]SELL CREATE, 1325.51
2012-06-22, SELL EXECUTED, 1325.92
2012-06-22, Close, 1335.02 ; Position 0.000000
2012-06-25, Close, 1313.72 ; Position 0.000000
2012-06-26, Close, 1319.99 ; Position 0.000000
2012-06-27, Close, 1331.85 ; Position 0.000000
2012-06-28, Close, 1329.04 ; Position 0.000000
2012-06-29, Close, 1362.16 ; Position 0.000000
2012-07-02, Close, 1365.51 ; Position 0.000000
2012-07-03, Close, 1374.02 ; Position 0.000000
2012-07-05, Close, 1367.58 ; Position 0.000000
2012-07-06, Close, 1354.68 ; Position 0.000000
2012-07-09, Close, 1352.46 ; Position 0.000000
2012-07-10, Close, 1341.47 ; Position 0.000000
2012-07-11, Close, 1341.45 ; Position 0.000000
2012-07-12, Close, 1334.76 ; Position 0.000000
2012-07-12, {mo}SELL CREATE, 1334.76
2012-07-13, SELL EXECUTED, 1334.81
2012-07-13, Close, 1356.78 ; Position -7.491984
2012-07-13, {mo}[Exit]BUY CREATE, 1356.78
2012-07-16, BUY EXECUTED, 1356.50
2012-07-16, Close, 1353.64 ; Position 0.000000
2012-07-17, Close, 13

2014-03-07, Close, 1878.04 ; Position 5.495260
2014-03-10, Close, 1877.17 ; Position 5.495260
2014-03-11, Close, 1867.63 ; Position 5.495260
2014-03-12, Close, 1868.20 ; Position 5.495260
2014-03-13, Close, 1846.34 ; Position 5.495260
2014-03-13, {mo}[Reverse]SELL CREATE, 1846.34
2014-03-14, SELL EXECUTED, 1845.07
2014-03-14, SELL EXECUTED, 1845.07
2014-03-14, Close, 1841.13 ; Position -5.416121
2014-03-17, Close, 1858.83 ; Position -5.416121
2014-03-17, {mo}[Exit]BUY CREATE, 1858.83
2014-03-18, BUY EXECUTED, 1858.92
2014-03-18, Close, 1872.25 ; Position 0.000000
2014-03-19, Close, 1860.77 ; Position 0.000000
2014-03-20, Close, 1872.01 ; Position 0.000000
2014-03-21, Close, 1866.52 ; Position 0.000000
2014-03-24, Close, 1857.44 ; Position 0.000000
2014-03-25, Close, 1865.62 ; Position 0.000000
2014-03-26, Close, 1852.56 ; Position 0.000000
2014-03-27, Close, 1849.04 ; Position 0.000000
2014-03-28, Close, 1857.62 ; Position 0.000000
2014-03-31, Close, 1872.34 ; Position 0.000000
2014-04

2015-09-08, Close, 1969.41 ; Position 0.000000
2015-09-09, Close, 1942.04 ; Position 0.000000
2015-09-10, Close, 1952.29 ; Position 0.000000
2015-09-10, {mo}BUY CREATE, 1952.29
2015-09-11, BUY EXECUTED, 1951.45
2015-09-11, Close, 1961.05 ; Position 5.122190
2015-09-14, Close, 1953.03 ; Position 5.122190
2015-09-15, Close, 1978.09 ; Position 5.122190
2015-09-16, Close, 1995.31 ; Position 5.122190
2015-09-17, Close, 1990.20 ; Position 5.122190
2015-09-18, Close, 1958.03 ; Position 5.122190
2015-09-18, {mo}[Exit]SELL CREATE, 1958.03
2015-09-21, SELL EXECUTED, 1960.84
2015-09-21, Close, 1966.97 ; Position 0.000000
2015-09-22, Close, 1942.74 ; Position 0.000000
2015-09-23, Close, 1938.76 ; Position 0.000000
2015-09-24, Close, 1932.24 ; Position 0.000000
2015-09-25, Close, 1931.34 ; Position 0.000000
2015-09-28, Close, 1881.77 ; Position 0.000000
2015-09-29, Close, 1884.09 ; Position 0.000000
2015-09-30, Close, 1920.03 ; Position 0.000000
2015-10-01, Close, 1923.82 ; Position 0.000000
2015-1

2017-05-22, BUY EXECUTED, 2387.21
2017-05-22, Close, 2394.02 ; Position 0.000000
2017-05-23, Close, 2398.42 ; Position 0.000000
2017-05-24, Close, 2404.39 ; Position 0.000000
2017-05-25, Close, 2415.07 ; Position 0.000000
2017-05-26, Close, 2415.82 ; Position 0.000000
2017-05-30, Close, 2412.91 ; Position 0.000000
2017-05-31, Close, 2411.80 ; Position 0.000000
2017-06-01, Close, 2430.06 ; Position 0.000000
2017-06-02, Close, 2439.07 ; Position 0.000000
2017-06-05, Close, 2436.10 ; Position 0.000000
2017-06-06, Close, 2429.33 ; Position 0.000000
2017-06-07, Close, 2433.14 ; Position 0.000000
2017-06-08, Close, 2433.79 ; Position 0.000000
2017-06-09, Close, 2431.77 ; Position 0.000000
2017-06-12, Close, 2429.39 ; Position 0.000000
2017-06-13, Close, 2440.35 ; Position 0.000000
2017-06-14, Close, 2437.92 ; Position 0.000000
2017-06-15, Close, 2432.46 ; Position 0.000000
2017-06-15, {mo}SELL CREATE, 2432.46
2017-06-16, SELL EXECUTED, 2431.24
2017-06-16, Close, 2433.15 ; Position -4.111065


## Mean-Reversion Strategy

In [83]:
# Create a Stratey
class MeanReversionStrategy(bt.Strategy):
    params = (
        ('period_BB', 15),
        ('devfactor', 1.2)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        
        # Add a MovingAverageSimple indicator
        self.midband = ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.period_BB)
        stddev = self.params.devfactor * bt.indicators.StdDev(self.datas[0], ma, period=self.params.period_BB,
                                           movav=bt.indicators.SimpleMovingAverage)
        self.topband = ma + stddev
        self.botband = ma - stddev
        
        # To keep track of pending orders
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def next(self):
        # Always long or short $10000 securities
        size = 10000./self.dataclose[0]
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f ; Position %f' % (self.dataclose[0],self.position.size))
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            #Open position
            if self.dataclose[0] <= self.botband[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('{mr}BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=size)

            if self.dataclose[0] >= self.topband[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('{mr}SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell(size=size)

        #Exit position
        elif self.position.size>0:
            #Reverse the long position
            if self.dataclose[0] >= self.topband[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('{mr}[Reverse]SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.close()
                self.order = self.sell(size=size)
                
            #Exit the long position
            elif self.dataclose[0] > self.midband[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('{mr}[Exit]SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()
                
        elif self.position.size<0:
            if self.dataclose[0] <= self.botband[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('{mr}[Reverse]BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.close()
                self.order = self.buy(size=size)

            
            #Exit the short position
            elif self.dataclose[0] < self.midband[0]:
                
                self.log('{mr}[Exit]BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.close()

run_backtest(MeanReversionStrategy,'mr')    

Starting Portfolio Value: 100000.00
2009-01-23, Close, 831.95 ; Position 0.000000
2009-01-26, Close, 836.57 ; Position 0.000000
2009-01-27, Close, 845.71 ; Position 0.000000
2009-01-28, Close, 874.09 ; Position 0.000000
2009-01-29, Close, 845.14 ; Position 0.000000
2009-01-30, Close, 825.88 ; Position 0.000000
2009-02-02, Close, 825.44 ; Position 0.000000
2009-02-03, Close, 838.51 ; Position 0.000000
2009-02-04, Close, 832.23 ; Position 0.000000
2009-02-05, Close, 845.85 ; Position 0.000000
2009-02-06, Close, 868.60 ; Position 0.000000
2009-02-06, {mr}SELL CREATE, 868.60
2009-02-09, SELL EXECUTED, 868.24
2009-02-09, Close, 869.89 ; Position -11.512779
2009-02-10, Close, 827.16 ; Position -11.512779
2009-02-10, {mr}[Exit]BUY CREATE, 827.16
2009-02-11, BUY EXECUTED, 827.41
2009-02-11, Close, 833.74 ; Position 0.000000
2009-02-12, Close, 835.19 ; Position 0.000000
2009-02-13, Close, 826.84 ; Position 0.000000
2009-02-17, Close, 789.17 ; Position 0.000000
2009-02-17, {mr}BUY CREATE, 789.17

2010-10-21, Close, 1180.26 ; Position -9.173470
2010-10-22, Close, 1183.08 ; Position -9.173470
2010-10-25, Close, 1185.62 ; Position -9.173470
2010-10-26, Close, 1185.64 ; Position -9.173470
2010-10-27, Close, 1182.45 ; Position -9.173470
2010-10-28, Close, 1183.78 ; Position -9.173470
2010-10-29, Close, 1183.26 ; Position -9.173470
2010-11-01, Close, 1184.38 ; Position -9.173470
2010-11-02, Close, 1193.57 ; Position -9.173470
2010-11-03, Close, 1197.96 ; Position -9.173470
2010-11-04, Close, 1221.06 ; Position -9.173470
2010-11-05, Close, 1225.85 ; Position -9.173470
2010-11-08, Close, 1223.25 ; Position -9.173470
2010-11-09, Close, 1213.40 ; Position -9.173470
2010-11-10, Close, 1218.71 ; Position -9.173470
2010-11-11, Close, 1213.54 ; Position -9.173470
2010-11-12, Close, 1199.21 ; Position -9.173470
2010-11-12, {mr}[Exit]BUY CREATE, 1199.21
2010-11-15, BUY EXECUTED, 1200.44
2010-11-15, Close, 1197.75 ; Position 0.000000
2010-11-16, Close, 1178.34 ; Position 0.000000
2010-11-16, {m

2012-06-28, Close, 1329.04 ; Position 0.000000
2012-06-29, Close, 1362.16 ; Position 0.000000
2012-06-29, {mr}SELL CREATE, 1362.16
2012-07-02, SELL EXECUTED, 1362.33
2012-07-02, Close, 1365.51 ; Position -7.341281
2012-07-03, Close, 1374.02 ; Position -7.341281
2012-07-05, Close, 1367.58 ; Position -7.341281
2012-07-06, Close, 1354.68 ; Position -7.341281
2012-07-09, Close, 1352.46 ; Position -7.341281
2012-07-10, Close, 1341.47 ; Position -7.341281
2012-07-10, {mr}[Exit]BUY CREATE, 1341.47
2012-07-11, BUY EXECUTED, 1341.40
2012-07-11, Close, 1341.45 ; Position 0.000000
2012-07-12, Close, 1334.76 ; Position 0.000000
2012-07-13, Close, 1356.78 ; Position 0.000000
2012-07-16, Close, 1353.64 ; Position 0.000000
2012-07-17, Close, 1363.67 ; Position 0.000000
2012-07-18, Close, 1372.78 ; Position 0.000000
2012-07-18, {mr}SELL CREATE, 1372.78
2012-07-19, SELL EXECUTED, 1373.01
2012-07-19, Close, 1376.51 ; Position -7.284488
2012-07-20, Close, 1362.66 ; Position -7.284488
2012-07-23, Close, 1

2014-02-07, Close, 1797.02 ; Position 5.585687
2014-02-07, {mr}[Exit]SELL CREATE, 1797.02
2014-02-10, SELL EXECUTED, 1796.20
2014-02-10, Close, 1799.84 ; Position 0.000000
2014-02-11, Close, 1819.75 ; Position 0.000000
2014-02-12, Close, 1819.26 ; Position 0.000000
2014-02-12, {mr}SELL CREATE, 1819.26
2014-02-13, SELL EXECUTED, 1814.82
2014-02-13, Close, 1829.83 ; Position -5.496740
2014-02-14, Close, 1838.63 ; Position -5.496740
2014-02-18, Close, 1840.76 ; Position -5.496740
2014-02-19, Close, 1828.75 ; Position -5.496740
2014-02-20, Close, 1839.78 ; Position -5.496740
2014-02-21, Close, 1836.25 ; Position -5.496740
2014-02-24, Close, 1847.61 ; Position -5.496740
2014-02-25, Close, 1845.12 ; Position -5.496740
2014-02-26, Close, 1845.16 ; Position -5.496740
2014-02-27, Close, 1854.29 ; Position -5.496740
2014-02-28, Close, 1859.45 ; Position -5.496740
2014-03-03, Close, 1845.73 ; Position -5.496740
2014-03-04, Close, 1873.91 ; Position -5.496740
2014-03-05, Close, 1873.81 ; Position 

2015-09-04, Close, 1921.22 ; Position 4.808594
2015-09-08, Close, 1969.41 ; Position 4.808594
2015-09-08, {mr}[Exit]SELL CREATE, 1969.41
2015-09-09, SELL EXECUTED, 1971.45
2015-09-09, Close, 1942.04 ; Position 0.000000
2015-09-10, Close, 1952.29 ; Position 0.000000
2015-09-11, Close, 1961.05 ; Position 0.000000
2015-09-14, Close, 1953.03 ; Position 0.000000
2015-09-15, Close, 1978.09 ; Position 0.000000
2015-09-16, Close, 1995.31 ; Position 0.000000
2015-09-16, {mr}SELL CREATE, 1995.31
2015-09-17, SELL EXECUTED, 1995.33
2015-09-17, Close, 1990.20 ; Position -5.011753
2015-09-18, Close, 1958.03 ; Position -5.011753
2015-09-18, {mr}[Exit]BUY CREATE, 1958.03
2015-09-21, BUY EXECUTED, 1960.84
2015-09-21, Close, 1966.97 ; Position 0.000000
2015-09-22, Close, 1942.74 ; Position 0.000000
2015-09-23, Close, 1938.76 ; Position 0.000000
2015-09-24, Close, 1932.24 ; Position 0.000000
2015-09-24, {mr}BUY CREATE, 1932.24
2015-09-25, BUY EXECUTED, 1935.93
2015-09-25, Close, 1931.34 ; Position 5.1753

2017-04-25, SELL EXECUTED, 2381.51
2017-04-25, Close, 2388.61 ; Position -4.212034
2017-04-26, Close, 2387.45 ; Position -4.212034
2017-04-27, Close, 2388.77 ; Position -4.212034
2017-04-28, Close, 2384.20 ; Position -4.212034
2017-05-01, Close, 2388.33 ; Position -4.212034
2017-05-02, Close, 2391.17 ; Position -4.212034
2017-05-03, Close, 2388.13 ; Position -4.212034
2017-05-04, Close, 2389.52 ; Position -4.212034
2017-05-05, Close, 2399.29 ; Position -4.212034
2017-05-08, Close, 2399.38 ; Position -4.212034
2017-05-09, Close, 2396.92 ; Position -4.212034
2017-05-10, Close, 2399.63 ; Position -4.212034
2017-05-11, Close, 2394.44 ; Position -4.212034
2017-05-12, Close, 2390.90 ; Position -4.212034
2017-05-15, Close, 2402.32 ; Position -4.212034
2017-05-16, Close, 2400.67 ; Position -4.212034
2017-05-17, Close, 2357.03 ; Position -4.212034
2017-05-17, {mr}[Reverse]BUY CREATE, 2357.03
2017-05-18, BUY EXECUTED, 2354.69
2017-05-18, BUY EXECUTED, 2354.69
2017-05-18, Close, 2365.72 ; Positio

## Features

In [19]:
def compute_hurst_exponent(arr):
    num_lag = 18
    lag_arr = np.arange(2,2+num_lag)
    tau_arr = np.zeros(num_lag)
    #  Step through the different lags
    for i in range(num_lag):
        lag = lag_arr[i]
        #  produce price difference with lag
        arr_diff = np.diff(arr,lag)
        #  Calculate the variance of the differnce vector
        tau_arr[i] = np.sqrt(np.std(arr_diff))
    #  linear fit to double-log graph (gives power)
    m = np.polyfit(np.log10(lag_arr),np.log10(tau_arr),1)
    # calculate hurst
    hurst = m[0]*2
    return hurst

def compute_vol(arr):
    log_arr = np.log(arr)
    return_arr = np.diff(log_arr)
    vol = np.float(np.sqrt(np.cov(return_arr)))
    return vol

def compute_long_short_scale_vol_ratio(arr, long_scale ,short_scale=1):
    arr_long_scale = arr[::long_scale]
    arr_short_scale = arr[::short_scale]
    
    return compute_vol(arr_short_scale)/compute_vol(arr_long_scale)

def compute_autocorr(arr, lag):
    log_arr = np.log(arr)
    return_arr = np.diff(log_arr)
    mat_autocov = np.cov(return_arr[lag:],return_arr[:-lag])
    autocorr = mat_autocov[1,0]/np.sqrt(mat_autocov[0,0]*mat_autocov[1,1])
    return autocorr

def compute_sharpe(arr):
    ret = np.divide(np.diff(arr),arr[:-1])
    return(np.mean(ret)/np.std(ret))

In [77]:
# Create a Stratey
class ComputeFeatureStrategy(bt.Strategy):
    params = (
        ('period_features', window_indicators),
        ('period_lookback', 3),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" and "volume" line in the data[0] dataseries
        self.pclose = self.datas[0].close
        self.volume = self.datas[0].volume
        ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.period_features)
        period_lookback = self.params.period_lookback
        
        # keep track of features
        self.hurst_exponent = deque([np.nan]*period_lookback)
        self.kurtosis = deque([np.nan]*period_lookback)
        self.long_short_scale_vol_ratio_3 = deque([np.nan]*period_lookback)
        self.long_short_scale_vol_ratio_4 = deque([np.nan]*period_lookback)
        self.long_short_scale_vol_ratio_5 = deque([np.nan]*period_lookback)
        self.autocorr_1 = deque([np.nan]*period_lookback)
        self.autocorr_2 = deque([np.nan]*period_lookback)
        self.autocorr_3 = deque([np.nan]*period_lookback)
        self.autocorr_4 = deque([np.nan]*period_lookback)
        self.autocorr_5 = deque([np.nan]*period_lookback)
        self.sharpe_ratio = deque([np.nan]*period_lookback)
        self.volume_change = deque([np.nan]*period_lookback)
        self.price_change = deque([np.nan]*period_lookback)
        self.pxv = deque([np.nan]*period_lookback)
         
        # To keep track of pending orders
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def next(self):
        
        # reference
        pclose_window = [self.pclose[i] for i in range(-self.params.period_features,0)]
        
        # Compute the features
        hurst_exponent = compute_hurst_exponent(pclose_window)
        kurtosis = sp.stats.kurtosis(pclose_window)
        long_short_scale_vol_ratio_3 = compute_long_short_scale_vol_ratio(pclose_window,3)
        long_short_scale_vol_ratio_4 = compute_long_short_scale_vol_ratio(pclose_window,4)
        long_short_scale_vol_ratio_5 = compute_long_short_scale_vol_ratio(pclose_window,5)
        autocorr_1 = compute_autocorr(pclose_window,1)
        autocorr_2 = compute_autocorr(pclose_window,2)
        autocorr_3 = compute_autocorr(pclose_window,3)
        autocorr_4 = compute_autocorr(pclose_window,4)
        autocorr_5 = compute_autocorr(pclose_window,5)
        sharpe_ratio = compute_sharpe(pclose_window)
        volume_change = self.volume[0] / self.volume[-1] - 1
        price_change = self.pclose[0] / self.pclose[-1] - 1
        pxv = volume_change * price_change
        
        # Record the features
        self.hurst_exponent.popleft()
        self.hurst_exponent.append(hurst_exponent)
        self.kurtosis.popleft()
        self.kurtosis.append(kurtosis)
        self.long_short_scale_vol_ratio_3.popleft()
        self.long_short_scale_vol_ratio_3.append(long_short_scale_vol_ratio_3)
        self.long_short_scale_vol_ratio_4.popleft()
        self.long_short_scale_vol_ratio_4.append(long_short_scale_vol_ratio_4)
        self.long_short_scale_vol_ratio_5.popleft()
        self.long_short_scale_vol_ratio_5.append(long_short_scale_vol_ratio_5)
        self.autocorr_1.popleft()
        self.autocorr_1.append(autocorr_1)
        self.autocorr_2.popleft()
        self.autocorr_2.append(autocorr_2)
        self.autocorr_3.popleft()
        self.autocorr_3.append(autocorr_3)
        self.autocorr_4.popleft()
        self.autocorr_4.append(autocorr_4)
        self.autocorr_5.popleft()
        self.autocorr_5.append(autocorr_5)
        self.sharpe_ratio.popleft()
        self.sharpe_ratio.append(sharpe_ratio)
        self.volume_change.popleft()
        self.volume_change.append(volume_change)
        self.price_change.popleft()
        self.price_change.append(price_change)
        self.pxv.popleft()
        self.pxv.append(pxv)
        
        self.log('Close, %.2f ; x %f' % (self.pclose[0],self.price_change[-1]))

run_backtest(ComputeFeatureStrategy,'feature')    

Starting Portfolio Value: 100000.00
2009-02-13, Close, 826.84 ; x -0.009998
2009-02-17, Close, 789.17 ; x -0.045559
2009-02-18, Close, 788.42 ; x -0.000950
2009-02-19, Close, 778.94 ; x -0.012024
2009-02-20, Close, 770.05 ; x -0.011413
2009-02-23, Close, 743.33 ; x -0.034699
2009-02-24, Close, 773.14 ; x 0.040103
2009-02-25, Close, 764.90 ; x -0.010658
2009-02-26, Close, 752.83 ; x -0.015780
2009-02-27, Close, 735.09 ; x -0.023564
2009-03-02, Close, 700.82 ; x -0.046620
2009-03-03, Close, 696.33 ; x -0.006407
2009-03-04, Close, 712.87 ; x 0.023753
2009-03-05, Close, 682.55 ; x -0.042532
2009-03-06, Close, 683.38 ; x 0.001216
2009-03-09, Close, 676.53 ; x -0.010024
2009-03-10, Close, 719.60 ; x 0.063663
2009-03-11, Close, 721.36 ; x 0.002446
2009-03-12, Close, 750.74 ; x 0.040729
2009-03-13, Close, 756.55 ; x 0.007739
2009-03-16, Close, 753.89 ; x -0.003516
2009-03-17, Close, 778.12 ; x 0.032140
2009-03-18, Close, 794.35 ; x 0.020858
2009-03-19, Close, 784.04 ; x -0.012979
2009-03-20, C

2010-02-17, Close, 1099.51 ; x 0.004238
2010-02-18, Close, 1106.75 ; x 0.006585
2010-02-19, Close, 1109.17 ; x 0.002187
2010-02-22, Close, 1108.01 ; x -0.001046
2010-02-23, Close, 1094.60 ; x -0.012103
2010-02-24, Close, 1105.24 ; x 0.009720
2010-02-25, Close, 1102.94 ; x -0.002081
2010-02-26, Close, 1104.49 ; x 0.001405
2010-03-01, Close, 1115.71 ; x 0.010159
2010-03-02, Close, 1118.31 ; x 0.002330
2010-03-03, Close, 1118.79 ; x 0.000429
2010-03-04, Close, 1122.97 ; x 0.003736
2010-03-05, Close, 1138.70 ; x 0.014007
2010-03-08, Close, 1138.50 ; x -0.000176
2010-03-09, Close, 1140.45 ; x 0.001713
2010-03-10, Close, 1145.61 ; x 0.004525
2010-03-11, Close, 1150.24 ; x 0.004042
2010-03-12, Close, 1149.99 ; x -0.000217
2010-03-15, Close, 1150.51 ; x 0.000452
2010-03-16, Close, 1159.46 ; x 0.007779
2010-03-17, Close, 1166.21 ; x 0.005822
2010-03-18, Close, 1165.83 ; x -0.000326
2010-03-19, Close, 1159.90 ; x -0.005087
2010-03-22, Close, 1165.81 ; x 0.005095
2010-03-23, Close, 1174.17 ; x 0.

2011-02-24, Close, 1306.10 ; x -0.000994
2011-02-25, Close, 1319.88 ; x 0.010550
2011-02-28, Close, 1327.22 ; x 0.005561
2011-03-01, Close, 1306.33 ; x -0.015740
2011-03-02, Close, 1308.44 ; x 0.001615
2011-03-03, Close, 1330.97 ; x 0.017219
2011-03-04, Close, 1321.15 ; x -0.007378
2011-03-07, Close, 1310.13 ; x -0.008341
2011-03-08, Close, 1321.82 ; x 0.008923
2011-03-09, Close, 1320.02 ; x -0.001362
2011-03-10, Close, 1295.11 ; x -0.018871
2011-03-11, Close, 1304.28 ; x 0.007080
2011-03-14, Close, 1296.39 ; x -0.006049
2011-03-15, Close, 1281.87 ; x -0.011200
2011-03-16, Close, 1256.88 ; x -0.019495
2011-03-17, Close, 1273.72 ; x 0.013398
2011-03-18, Close, 1279.21 ; x 0.004310
2011-03-21, Close, 1298.38 ; x 0.014986
2011-03-22, Close, 1293.77 ; x -0.003551
2011-03-23, Close, 1297.54 ; x 0.002914
2011-03-24, Close, 1309.66 ; x 0.009341
2011-03-25, Close, 1313.80 ; x 0.003161
2011-03-28, Close, 1310.19 ; x -0.002748
2011-03-29, Close, 1319.44 ; x 0.007060
2011-03-30, Close, 1328.26 ; 

2012-03-08, Close, 1365.91 ; x 0.009818
2012-03-09, Close, 1370.87 ; x 0.003631
2012-03-12, Close, 1371.09 ; x 0.000160
2012-03-13, Close, 1395.95 ; x 0.018132
2012-03-14, Close, 1394.28 ; x -0.001196
2012-03-15, Close, 1402.60 ; x 0.005967
2012-03-16, Close, 1404.17 ; x 0.001119
2012-03-19, Close, 1409.75 ; x 0.003974
2012-03-20, Close, 1405.52 ; x -0.003001
2012-03-21, Close, 1402.89 ; x -0.001871
2012-03-22, Close, 1392.78 ; x -0.007207
2012-03-23, Close, 1397.11 ; x 0.003109
2012-03-26, Close, 1416.51 ; x 0.013886
2012-03-27, Close, 1412.52 ; x -0.002817
2012-03-28, Close, 1405.54 ; x -0.004942
2012-03-29, Close, 1403.28 ; x -0.001608
2012-03-30, Close, 1408.47 ; x 0.003698
2012-04-02, Close, 1419.04 ; x 0.007505
2012-04-03, Close, 1413.38 ; x -0.003989
2012-04-04, Close, 1398.96 ; x -0.010202
2012-04-05, Close, 1398.08 ; x -0.000629
2012-04-09, Close, 1382.20 ; x -0.011358
2012-04-10, Close, 1358.59 ; x -0.017081
2012-04-11, Close, 1368.71 ; x 0.007449
2012-04-12, Close, 1387.57 ;

2013-03-22, Close, 1556.89 ; x 0.007174
2013-03-25, Close, 1551.69 ; x -0.003340
2013-03-26, Close, 1563.77 ; x 0.007785
2013-03-27, Close, 1562.85 ; x -0.000588
2013-03-28, Close, 1569.19 ; x 0.004057
2013-04-01, Close, 1562.17 ; x -0.004474
2013-04-02, Close, 1570.25 ; x 0.005172
2013-04-03, Close, 1553.69 ; x -0.010546
2013-04-04, Close, 1559.98 ; x 0.004048
2013-04-05, Close, 1553.28 ; x -0.004295
2013-04-08, Close, 1563.07 ; x 0.006303
2013-04-09, Close, 1568.61 ; x 0.003544
2013-04-10, Close, 1587.73 ; x 0.012189
2013-04-11, Close, 1593.37 ; x 0.003552
2013-04-12, Close, 1588.85 ; x -0.002837
2013-04-15, Close, 1552.36 ; x -0.022966
2013-04-16, Close, 1574.57 ; x 0.014307
2013-04-17, Close, 1552.01 ; x -0.014328
2013-04-18, Close, 1541.61 ; x -0.006701
2013-04-19, Close, 1555.25 ; x 0.008848
2013-04-22, Close, 1562.50 ; x 0.004662
2013-04-23, Close, 1578.78 ; x 0.010419
2013-04-24, Close, 1578.79 ; x 0.000006
2013-04-25, Close, 1585.16 ; x 0.004035
2013-04-26, Close, 1582.24 ; x 

2014-04-04, Close, 1865.09 ; x -0.012537
2014-04-07, Close, 1845.04 ; x -0.010750
2014-04-08, Close, 1851.96 ; x 0.003751
2014-04-09, Close, 1872.18 ; x 0.010918
2014-04-10, Close, 1833.08 ; x -0.020885
2014-04-11, Close, 1815.69 ; x -0.009487
2014-04-14, Close, 1830.61 ; x 0.008217
2014-04-15, Close, 1842.98 ; x 0.006757
2014-04-16, Close, 1862.31 ; x 0.010488
2014-04-17, Close, 1864.85 ; x 0.001364
2014-04-21, Close, 1871.89 ; x 0.003775
2014-04-22, Close, 1879.55 ; x 0.004092
2014-04-23, Close, 1875.39 ; x -0.002213
2014-04-24, Close, 1878.61 ; x 0.001717
2014-04-25, Close, 1863.40 ; x -0.008096
2014-04-28, Close, 1869.43 ; x 0.003236
2014-04-29, Close, 1878.33 ; x 0.004761
2014-04-30, Close, 1883.95 ; x 0.002992
2014-05-01, Close, 1883.68 ; x -0.000143
2014-05-02, Close, 1881.14 ; x -0.001348
2014-05-05, Close, 1884.66 ; x 0.001871
2014-05-06, Close, 1867.72 ; x -0.008988
2014-05-07, Close, 1878.21 ; x 0.005616
2014-05-08, Close, 1875.63 ; x -0.001374
2014-05-09, Close, 1878.48 ; x

2015-04-23, Close, 2112.93 ; x 0.002358
2015-04-24, Close, 2117.69 ; x 0.002253
2015-04-27, Close, 2108.92 ; x -0.004141
2015-04-28, Close, 2114.76 ; x 0.002769
2015-04-29, Close, 2106.85 ; x -0.003740
2015-04-30, Close, 2085.51 ; x -0.010129
2015-05-01, Close, 2108.29 ; x 0.010923
2015-05-04, Close, 2114.49 ; x 0.002941
2015-05-05, Close, 2089.46 ; x -0.011837
2015-05-06, Close, 2080.15 ; x -0.004456
2015-05-07, Close, 2088.00 ; x 0.003774
2015-05-08, Close, 2116.10 ; x 0.013458
2015-05-11, Close, 2105.33 ; x -0.005090
2015-05-12, Close, 2099.12 ; x -0.002950
2015-05-13, Close, 2098.48 ; x -0.000305
2015-05-14, Close, 2121.10 ; x 0.010779
2015-05-15, Close, 2122.73 ; x 0.000768
2015-05-18, Close, 2129.20 ; x 0.003048
2015-05-19, Close, 2127.83 ; x -0.000643
2015-05-20, Close, 2125.85 ; x -0.000931
2015-05-21, Close, 2130.82 ; x 0.002338
2015-05-22, Close, 2126.06 ; x -0.002234
2015-05-26, Close, 2104.20 ; x -0.010282
2015-05-27, Close, 2123.48 ; x 0.009163
2015-05-28, Close, 2120.79 ;

2016-05-06, Close, 2057.14 ; x 0.003175
2016-05-09, Close, 2058.69 ; x 0.000753
2016-05-10, Close, 2084.39 ; x 0.012484
2016-05-11, Close, 2064.46 ; x -0.009562
2016-05-12, Close, 2064.11 ; x -0.000170
2016-05-13, Close, 2046.61 ; x -0.008478
2016-05-16, Close, 2066.66 ; x 0.009797
2016-05-17, Close, 2047.21 ; x -0.009411
2016-05-18, Close, 2047.63 ; x 0.000205
2016-05-19, Close, 2040.04 ; x -0.003707
2016-05-20, Close, 2052.32 ; x 0.006019
2016-05-23, Close, 2048.04 ; x -0.002085
2016-05-24, Close, 2076.06 ; x 0.013681
2016-05-25, Close, 2090.54 ; x 0.006975
2016-05-26, Close, 2090.10 ; x -0.000210
2016-05-27, Close, 2099.06 ; x 0.004287
2016-05-31, Close, 2096.95 ; x -0.001005
2016-06-01, Close, 2099.33 ; x 0.001135
2016-06-02, Close, 2105.26 ; x 0.002825
2016-06-03, Close, 2099.13 ; x -0.002912
2016-06-06, Close, 2109.41 ; x 0.004897
2016-06-07, Close, 2112.13 ; x 0.001289
2016-06-08, Close, 2119.12 ; x 0.003309
2016-06-09, Close, 2115.48 ; x -0.001718
2016-06-10, Close, 2096.07 ; x

2017-05-15, Close, 2402.32 ; x 0.004776
2017-05-16, Close, 2400.67 ; x -0.000687
2017-05-17, Close, 2357.03 ; x -0.018178
2017-05-18, Close, 2365.72 ; x 0.003687
2017-05-19, Close, 2381.73 ; x 0.006767
2017-05-22, Close, 2394.02 ; x 0.005160
2017-05-23, Close, 2398.42 ; x 0.001838
2017-05-24, Close, 2404.39 ; x 0.002489
2017-05-25, Close, 2415.07 ; x 0.004442
2017-05-26, Close, 2415.82 ; x 0.000311
2017-05-30, Close, 2412.91 ; x -0.001205
2017-05-31, Close, 2411.80 ; x -0.000460
2017-06-01, Close, 2430.06 ; x 0.007571
2017-06-02, Close, 2439.07 ; x 0.003708
2017-06-05, Close, 2436.10 ; x -0.001218
2017-06-06, Close, 2429.33 ; x -0.002779
2017-06-07, Close, 2433.14 ; x 0.001568
2017-06-08, Close, 2433.79 ; x 0.000267
2017-06-09, Close, 2431.77 ; x -0.000830
2017-06-12, Close, 2429.39 ; x -0.000979
2017-06-13, Close, 2440.35 ; x 0.004511
2017-06-14, Close, 2437.92 ; x -0.000996
2017-06-15, Close, 2432.46 ; x -0.002240
2017-06-16, Close, 2433.15 ; x 0.000284
2017-06-19, Close, 2453.46 ; x

## Training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Load data
df = pd.read_csv("../../../../datas/spx-1950-2018.csv")
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
Price = df.loc["2000-01-01":"2018-02-09","Close"]
Volume = df.loc["2000-01-01":"2018-02-09","Volume"]

In [79]:
window_indicators = 30
Indicators = pd.DataFrame(index = Price.index, columns=["hurst_exponent",
                                                        "kurtosis",
                                                        "long_short_scale_vol_ratio_3",
                                                        "long_short_scale_vol_ratio_4",
                                                        "long_short_scale_vol_ratio_5",
                                                        "autocorr_1",
                                                        "autocorr_2",
                                                        "autocorr_3",
                                                        "autocorr_4",
                                                        "autocorr_5",
                                                        "sharpe_ratio",
                                                        "volume_change",
                                                        "price_change",
                                                        "pxv"])
Indicators.loc[:,"hurst_exponent"] = Price.rolling(window=window_indicators).apply(compute_hurst_exponent)
Indicators.loc[:,"kurtosis"] = Price.rolling(window=window_indicators).apply(lambda x: sp.stats.kurtosis(x))
Indicators.loc[:,"long_short_scale_vol_ratio_3"] = Price.rolling(window=window_indicators).apply(lambda x:compute_long_short_scale_vol_ratio(x,3))
Indicators.loc[:,"long_short_scale_vol_ratio_4"] = Price.rolling(window=window_indicators).apply(lambda x:compute_long_short_scale_vol_ratio(x,4))
Indicators.loc[:,"long_short_scale_vol_ratio_5"] = Price.rolling(window=window_indicators).apply(lambda x:compute_long_short_scale_vol_ratio(x,5))
Indicators.loc[:,"autocorr_1"] = Price.rolling(window=window_indicators).apply(lambda x:compute_autocorr(x,1))
Indicators.loc[:,"autocorr_2"] = Price.rolling(window=window_indicators).apply(lambda x:compute_autocorr(x,2))
Indicators.loc[:,"autocorr_3"] = Price.rolling(window=window_indicators).apply(lambda x:compute_autocorr(x,3))
Indicators.loc[:,"autocorr_4"] = Price.rolling(window=window_indicators).apply(lambda x:compute_autocorr(x,4))
Indicators.loc[:,"autocorr_5"] = Price.rolling(window=window_indicators).apply(lambda x:compute_autocorr(x,5))
Indicators.loc[:,"sharpe_ratio"] = Price.rolling(window=window_indicators).apply(compute_sharpe)
Indicators.loc[:,"volume_change"] = Volume.diff()/Volume.shift()
Indicators.loc[:,"price_change"] =  Price.diff()/Price.shift()
Indicators.loc[:,"pxv"] = Indicators.loc[:,"volume_change"] * Indicators.loc[:,"price_change"]
df_dropna = Indicators.dropna()

In [141]:
df_mo = pd.read_csv("mo.csv")
df_mo = df_mo.loc[df_mo.Id.apply(lambda s:s.isdigit())]
df_mo.set_index(df_mo.datetime.apply(lambda x : pd.to_datetime(x[:10])),inplace=True)

In [136]:
df_mr = pd.read_csv("mr.csv")
df_mr = df_mr.loc[df_mr.Id.apply(lambda s:s.isdigit())]
df_mr.set_index(df_mr.datetime.apply(lambda x : pd.to_datetime(x[:10])),inplace=True)

In [137]:
# Parameters
window_lookback = 3
window_prediction = 9
num_features = 14

In [138]:
# Feature
df_dropna = df_dropna.iloc[:,:num_features].dropna()
X = np.zeros((len(df_dropna.index),num_features*window_lookback))
idx_X = [None]*X.shape[0]
for i in range(window_lookback-1,len(df_dropna.index)):
    # transform the feature matrix for a certain window into a vector
    X[i,:] = df_dropna.iloc[i-window_lookback+1:i+1,:num_features].values.reshape(1,-1)
    idx_X[i] = df_dropna.index[i]
X = pd.DataFrame(X[window_lookback-1:],index=idx_X[window_lookback-1:])

In [150]:
return_mo = df_mo.value.shift(-1)/df_mo.value-1.
return_mr = df_mr.value.shift(-1)/df_mr.value-1.

In [172]:
# Label
mo_minus_mr =  return_mo - return_mr
y = mo_minus_mr.rolling(window=window_prediction).mean().shift(-window_prediction+1)

In [177]:
# Align the Features and Labels
# index
idx = X.dropna().index.intersection(y.dropna().index)
# Feature
X = X.loc[idx,:]
# Label
y_class = pd.Series(0,index = y.index,dtype=int)
y_class[y>=0] =+1
y_class.loc[(return_mo<-0.0003) & (return_mr<-0.0003)] = 0
y_class[y<=-0] =-1
y_class = y_class.loc[idx]

In [181]:
# split to train and test
X_train, X_test = X["2000-01-01":"2013-08-13"], X["2013-08-14":]
y_train, y_test = y_class["2000-01-01":"2013-08-13"], y_class["2013-08-14":]
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
# Train
rnd_clf.fit(X_train, y_train)
train_score = rnd_clf.score(X_train, y_train)
print("Accuracy in train set: %.4f%%"%(train_score*100))
# Test
test_score = rnd_clf.score(X_test, y_test)
print("Accuracy in test set: %.4f%%"%(test_score*100))

Accuracy in train set: 83.8053%
Accuracy in test set: 54.0107%


In [192]:
def train_rf(start_year,end_year):
    """ Train a random forest with data between start_year (inclusive) and end_year (exclusive)
    """
    start = str(start_year)+"-01-01"
    end = str(end_year)+"-01-01"
    X_train = X[start:end]
    y_train = y_class[start:end]
    rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
    rnd_clf.fit(X_train, y_train)
    train_score = rnd_clf.score(X_train, y_train)
    print("Accuracy in train set: %.4f%%"%(train_score*100))
    return rnd_clf

## Regime-Switching Strategy

In [187]:
def generate_RegimeSwitchingStrategy(rnd_clf):
    class RegimeSwitchingStrategy(bt.Strategy):
        '''
        This strategy is loosely based on some of the examples from the Van
        K. Tharp book: *Trade Your Way To Financial Freedom*. The logic:

          - Enter the market if:
            - The MACD.macd line crosses the MACD.signal line to the upside
            - The Simple Moving Average has a negative direction in the last x
              periods (actual value below value x periods ago)

         - Set a stop price x times the ATR value away from the close

         - If in the market:

           - Check if the current close has gone below the stop price. If yes,
             exit.
           - If not, update the stop price if the new stop price would be higher
             than the current
        '''

        params = (
            # Momentum MACD Parameters
            ('macd1', 12),
            ('macd2', 26),
            ('macdsig', 9),
            ('atrperiod', 14),  # ATR Period (standard)
            ('atrdist', 1),   # ATR distance for stop price
            ('smaperiod', 30),  # SMA Period (pretty standard)
            ('dirperiod', 10),  # Lookback period to consider SMA trend direction
            # Mean Reversion Parameters
            ('period_BB', 15),
            ('devfactor', 1.2),
            ('period_features', 30),
            ('period_lookback', 3),
        )

        def log(self, txt, dt=None):
            ''' Logging function fot this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log('BUY EXECUTED, %.2f' % order.executed.price)
                elif order.issell():
                    self.log('SELL EXECUTED, %.2f' % order.executed.price)

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            # Write down: no pending order
            self.order = None

        def __init__(self):

            # Keep a reference to the "close" line in the data[0] dataseries
            self.dataclose = self.datas[0].close
            self.pclose = self.datas[0].close
            self.volume = self.datas[0].volume
            period_lookback = self.params.period_lookback
            
            # Keep track of features
            self.hurst_exponent = deque([np.nan]*period_lookback)
            self.kurtosis = deque([np.nan]*period_lookback)
            self.long_short_scale_vol_ratio_3 = deque([np.nan]*period_lookback)
            self.long_short_scale_vol_ratio_4 = deque([np.nan]*period_lookback)
            self.long_short_scale_vol_ratio_5 = deque([np.nan]*period_lookback)
            self.autocorr_1 = deque([np.nan]*period_lookback)
            self.autocorr_2 = deque([np.nan]*period_lookback)
            self.autocorr_3 = deque([np.nan]*period_lookback)
            self.autocorr_4 = deque([np.nan]*period_lookback)
            self.autocorr_5 = deque([np.nan]*period_lookback)
            self.sharpe_ratio = deque([np.nan]*period_lookback)
            self.volume_change = deque([np.nan]*period_lookback)
            self.price_change = deque([np.nan]*period_lookback)
            self.pxv = deque([np.nan]*period_lookback)

            ##Momentum

            # MACD Indicator
            self.macd = bt.indicators.MACD(self.datas[0],
                                           period_me1=self.p.macd1,
                                           period_me2=self.p.macd2,
                                           period_signal=self.p.macdsig)

            # Cross of macd.macd and macd.signal
            self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)
            self.mcross2 = bt.indicators.CrossDown(self.macd.macd, self.macd.signal)
            # To set the stop price
            self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)

            # Control market trend
            self.sma = bt.indicators.SMA(self.data, period=self.p.smaperiod)
            self.smadir = self.sma - self.sma(-self.p.dirperiod)

            ##Mean-Reversion
            # Bollinger Band
            self.midband = ma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.period_BB)
            stddev = self.params.devfactor * bt.indicators.StdDev(self.datas[0], ma, period=self.params.period_BB,
                                               movav=bt.indicators.SimpleMovingAverage)
            self.topband = ma + stddev
            self.botband = ma - stddev

        def start(self):
            self.order = None  # sentinel to avoid operrations on pending order

        def next(self):
            # Always long or short $10000 securities
            size = 10000./self.dataclose[0]

            # Simply log the closing price of the series from the reference
            self.log('Close, %.2f ; Position %f' % (self.dataclose[0],self.position.size))

            if self.order:
                return  # pending order execution
            ##Feature Part
            # reference
            pclose_window = [self.pclose[i] for i in range(-self.params.period_features,0)]

            # Compute the features
            hurst_exponent = compute_hurst_exponent(pclose_window)
            kurtosis = sp.stats.kurtosis(pclose_window)
            long_short_scale_vol_ratio_3 = compute_long_short_scale_vol_ratio(pclose_window,3)
            long_short_scale_vol_ratio_4 = compute_long_short_scale_vol_ratio(pclose_window,4)
            long_short_scale_vol_ratio_5 = compute_long_short_scale_vol_ratio(pclose_window,5)
            autocorr_1 = compute_autocorr(pclose_window,1)
            autocorr_2 = compute_autocorr(pclose_window,2)
            autocorr_3 = compute_autocorr(pclose_window,3)
            autocorr_4 = compute_autocorr(pclose_window,4)
            autocorr_5 = compute_autocorr(pclose_window,5)
            sharpe_ratio = compute_sharpe(pclose_window)
            volume_change = self.volume[0] / self.volume[-1] - 1
            price_change = self.pclose[0] / self.pclose[-1] - 1
            pxv = volume_change * price_change

            # Record the features
            self.hurst_exponent.popleft()
            self.hurst_exponent.append(hurst_exponent)
            self.kurtosis.popleft()
            self.kurtosis.append(kurtosis)
            self.long_short_scale_vol_ratio_3.popleft()
            self.long_short_scale_vol_ratio_3.append(long_short_scale_vol_ratio_3)
            self.long_short_scale_vol_ratio_4.popleft()
            self.long_short_scale_vol_ratio_4.append(long_short_scale_vol_ratio_4)
            self.long_short_scale_vol_ratio_5.popleft()
            self.long_short_scale_vol_ratio_5.append(long_short_scale_vol_ratio_5)
            self.autocorr_1.popleft()
            self.autocorr_1.append(autocorr_1)
            self.autocorr_2.popleft()
            self.autocorr_2.append(autocorr_2)
            self.autocorr_3.popleft()
            self.autocorr_3.append(autocorr_3)
            self.autocorr_4.popleft()
            self.autocorr_4.append(autocorr_4)
            self.autocorr_5.popleft()
            self.autocorr_5.append(autocorr_5)
            self.sharpe_ratio.popleft()
            self.sharpe_ratio.append(sharpe_ratio)
            self.volume_change.popleft()
            self.volume_change.append(volume_change)
            self.price_change.popleft()
            self.price_change.append(price_change)
            self.pxv.popleft()
            self.pxv.append(pxv)

            #Construct Feature Vector
            feature_list = [self.hurst_exponent,
                            self.kurtosis,
                            self.long_short_scale_vol_ratio_3,
                            self.long_short_scale_vol_ratio_4,
                            self.long_short_scale_vol_ratio_5,
                            self.autocorr_1,
                            self.autocorr_2,
                            self.autocorr_3,
                            self.autocorr_4,
                            self.autocorr_5,
                            self.sharpe_ratio,
                            self.volume_change,
                            self.price_change,
                            self.pxv]
            fv = np.array([list(f) for f in feature_list]).T.reshape(1,-1)
            if np.any(np.isnan(fv)):
                self.regime = 0
            else:
                self.regime = rnd_clf.predict(fv)

            #Momentum
            if np.allclose(self.regime,1): 
                if not self.position:  # not in the market
                    if self.mcross[0] > 0.0 and self.smadir < 0.0 and self.macd.macd[0]>=self.macd.macd[-1]:
                        self.order = self.buy(size=size)
                        self.log('{mo}BUY CREATE, %.2f' % self.dataclose[0])
                        pdist = self.atr[0] * self.p.atrdist
                        self.pstop = self.data.close[0] - pdist

                    elif self.mcross[0] < 0.0 and self.smadir > 0.0 and self.macd.macd[0]<=self.macd.macd[-1]:
                        self.order = self.sell(size=size)
                        self.log('{mo}SELL CREATE, %.2f' % self.dataclose[0])
                        pdist = self.atr[0] * self.p.atrdist
                        self.pstop = self.data.close[0] + pdist


                else:  # in the market
                    if self.position.size>0:
                        pclose = self.data.close[0]
                        pstop = self.pstop
                        pdist = self.atr[0] * self.p.atrdist

                        # Reverse long position
                        if self.mcross[0] < 0.0 and self.smadir > 0.0 and self.macd.macd[0]<=self.macd.macd[-1]:
                            self.close()
                            self.order = self.sell(size=size)
                            self.log('{mo}[Reverse]SELL CREATE, %.2f' % self.dataclose[0])
                            self.pstop = self.data.close[0] + pdist

                        # Close long position
                        elif pclose < pstop:
                            self.close()  # stop met - get out
                            self.log('{mo}[Exit]SELL CREATE, %.2f' % self.dataclose[0])
                        else:
                            # Update only if greater than
                            self.pstop = max(pstop, pclose - pdist)

                    elif self.position.size<0:
                        pclose = self.data.close[0]
                        pstop = self.pstop
                        pdist = self.atr[0] * self.p.atrdist

                        # Reverse short position
                        if self.mcross[0] > 0.0 and self.smadir < 0.0 and self.macd.macd[0]>=self.macd.macd[-1]:
                            self.order = self.buy(size=size)
                            self.log('{mo}[Reverse]BUY CREATE, %.2f' % self.dataclose[0])
                            pdist = self.atr[0] * self.p.atrdist
                            self.pstop = self.data.close[0] - pdist

                        # Close short position
                        if pclose > pstop:
                            self.close()  # stop met - get out
                            self.log('{mo}[Exit]BUY CREATE, %.2f' % self.dataclose[0])
                        else:
                            # Update only if greater than
                            self.pstop = min(pstop, pclose + pdist)

            #Mean-Reversion
            elif np.allclose(self.regime,-1): 
                # Check if we are in the market
                if not self.position:
                    #Open position
                    if self.dataclose[0] <= self.botband[0]:

                        # BUY, BUY, BUY!!! (with all possible default parameters)
                        self.log('{mr}BUY CREATE, %.2f' % self.dataclose[0])
                        pdist = self.atr[0] * self.p.atrdist
                        self.pstop = self.data.close[0] - pdist
                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy(size=size)

                    if self.dataclose[0] >= self.topband[0]:
                        # SELL, SELL, SELL!!! (with all possible default parameters)
                        self.log('{mr}SELL CREATE, %.2f' % self.dataclose[0])
                        pdist = self.atr[0] * self.p.atrdist
                        self.pstop = self.data.close[0] + pdist
                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.sell(size=size)

                #Exit position
                elif self.position.size>0:
                    #Reverse the long position
                    if self.dataclose[0] >= self.topband[0]:
                        # SELL, SELL, SELL!!! (with all possible default parameters)
                        self.log('{mr}[Reverse]SELL CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.close()
                        self.order = self.sell(size=size)

                    #Exit the long position
                    elif self.dataclose[0] > self.midband[0]:
                        # SELL, SELL, SELL!!! (with all possible default parameters)
                        self.log('{mr}[Exit]SELL CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.close()

                elif self.position.size<0:
                    if self.dataclose[0] <= self.botband[0]:

                        # BUY, BUY, BUY!!! (with all possible default parameters)
                        self.log('{mr}[Reverse]BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.close()
                        self.order = self.buy(size=size)


                    #Exit the short position
                    elif self.dataclose[0] < self.midband[0]:

                        self.log('{mr}[Exit]BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.close()
    return RegimeSwitchingStrategy

## Walk-Forward Train and Test

In [194]:
rnd_clf = train_rf(start_year=2009,end_year=2012)
RegimeSwitchingStrategy = generate_RegimeSwitchingStrategy(rnd_clf)
run_backtest(RegimeSwitchingStrategy,year_start=2012,year_end=2018,label="s")

Accuracy in train set: 86.8966%
Starting Portfolio Value: 100000.00
2012-02-21, Close, 1362.21 ; Position 0.000000
2012-02-22, Close, 1357.66 ; Position 0.000000
2012-02-23, Close, 1363.46 ; Position 0.000000
2012-02-23, {mr}SELL CREATE, 1363.46
2012-02-24, SELL EXECUTED, 1363.46
2012-02-24, Close, 1365.74 ; Position -7.334282
2012-02-27, Close, 1367.59 ; Position -7.334282
2012-02-28, Close, 1372.18 ; Position -7.334282
2012-02-29, Close, 1365.68 ; Position -7.334282
2012-03-01, Close, 1374.09 ; Position -7.334282
2012-03-02, Close, 1369.63 ; Position -7.334282
2012-03-05, Close, 1364.33 ; Position -7.334282
2012-03-06, Close, 1343.36 ; Position -7.334282
2012-03-06, {mr}[Reverse]BUY CREATE, 1343.36
2012-03-07, BUY EXECUTED, 1343.39
2012-03-07, BUY EXECUTED, 1343.39
2012-03-07, Close, 1352.63 ; Position 7.444021
2012-03-07, {mo}[Exit]SELL CREATE, 1352.63
2012-03-08, SELL EXECUTED, 1352.65
2012-03-08, Close, 1365.91 ; Position 0.000000
2012-03-09, Close, 1370.87 ; Position 0.000000
201

2012-09-19, Close, 1461.05 ; Position -6.822353
2012-09-20, Close, 1460.26 ; Position -6.822353
2012-09-21, Close, 1460.15 ; Position -6.822353
2012-09-24, Close, 1456.89 ; Position -6.822353
2012-09-25, Close, 1441.59 ; Position -6.822353
2012-09-26, Close, 1433.32 ; Position -6.822353
2012-09-27, Close, 1447.15 ; Position -6.822353
2012-09-27, {mo}[Exit]BUY CREATE, 1447.15
2012-09-28, BUY EXECUTED, 1447.13
2012-09-28, Close, 1440.67 ; Position 0.000000
2012-10-01, Close, 1444.49 ; Position 0.000000
2012-10-02, Close, 1445.75 ; Position 0.000000
2012-10-03, Close, 1450.99 ; Position 0.000000
2012-10-04, Close, 1461.40 ; Position 0.000000
2012-10-05, Close, 1460.93 ; Position 0.000000
2012-10-08, Close, 1455.88 ; Position 0.000000
2012-10-09, Close, 1441.48 ; Position 0.000000
2012-10-10, Close, 1432.56 ; Position 0.000000
2012-10-11, Close, 1432.84 ; Position 0.000000
2012-10-12, Close, 1428.59 ; Position 0.000000
2012-10-15, Close, 1440.13 ; Position 0.000000
2012-10-16, Close, 1454.

2013-04-17, Close, 1552.01 ; Position 0.000000
2013-04-18, Close, 1541.61 ; Position 0.000000
2013-04-18, {mr}BUY CREATE, 1541.61
2013-04-19, BUY EXECUTED, 1541.61
2013-04-19, Close, 1555.25 ; Position 6.486725
2013-04-22, Close, 1562.50 ; Position 6.486725
2013-04-23, Close, 1578.78 ; Position 6.486725
2013-04-23, {mr}[Exit]SELL CREATE, 1578.78
2013-04-24, SELL EXECUTED, 1578.78
2013-04-24, Close, 1578.79 ; Position 0.000000
2013-04-25, Close, 1585.16 ; Position 0.000000
2013-04-26, Close, 1582.24 ; Position 0.000000
2013-04-29, Close, 1593.61 ; Position 0.000000
2013-04-29, {mr}SELL CREATE, 1593.61
2013-04-30, SELL EXECUTED, 1593.58
2013-04-30, Close, 1597.57 ; Position -6.275061
2013-05-01, Close, 1582.70 ; Position -6.275061
2013-05-02, Close, 1597.59 ; Position -6.275061
2013-05-03, Close, 1614.42 ; Position -6.275061
2013-05-03, {mo}[Exit]BUY CREATE, 1614.42
2013-05-06, BUY EXECUTED, 1614.40
2013-05-06, Close, 1617.50 ; Position 0.000000
2013-05-06, {mr}SELL CREATE, 1617.50
2013-

2013-10-25, Close, 1759.77 ; Position 0.000000
2013-10-28, Close, 1762.11 ; Position 0.000000
2013-10-29, Close, 1771.95 ; Position 0.000000
2013-10-30, Close, 1763.31 ; Position 0.000000
2013-10-31, Close, 1756.54 ; Position 0.000000
2013-11-01, Close, 1761.64 ; Position 0.000000
2013-11-04, Close, 1767.93 ; Position 0.000000
2013-11-05, Close, 1762.97 ; Position 0.000000
2013-11-06, Close, 1770.49 ; Position 0.000000
2013-11-07, Close, 1747.15 ; Position 0.000000
2013-11-07, {mo}SELL CREATE, 1747.15
2013-11-08, SELL EXECUTED, 1748.37
2013-11-08, Close, 1770.61 ; Position -5.723607
2013-11-08, {mo}[Exit]BUY CREATE, 1770.61
2013-11-11, BUY EXECUTED, 1769.96
2013-11-11, Close, 1771.89 ; Position 0.000000
2013-11-12, Close, 1767.69 ; Position 0.000000
2013-11-13, Close, 1782.00 ; Position 0.000000
2013-11-14, Close, 1790.62 ; Position 0.000000
2013-11-15, Close, 1798.18 ; Position 0.000000
2013-11-15, {mr}SELL CREATE, 1798.18
2013-11-18, SELL EXECUTED, 1798.82
2013-11-18, Close, 1791.53 

2014-05-12, {mo}[Exit]BUY CREATE, 1896.65
2014-05-13, BUY EXECUTED, 1896.75
2014-05-13, Close, 1897.45 ; Position 0.000000
2014-05-14, Close, 1888.53 ; Position 0.000000
2014-05-15, Close, 1870.85 ; Position 0.000000
2014-05-15, {mo}SELL CREATE, 1870.85
2014-05-16, SELL EXECUTED, 1871.19
2014-05-16, Close, 1877.86 ; Position -5.345164
2014-05-19, Close, 1885.08 ; Position -5.345164
2014-05-20, Close, 1872.83 ; Position -5.345164
2014-05-21, Close, 1888.03 ; Position -5.345164
2014-05-21, {mo}[Exit]BUY CREATE, 1888.03
2014-05-22, BUY EXECUTED, 1888.19
2014-05-22, Close, 1892.49 ; Position 0.000000
2014-05-23, Close, 1900.53 ; Position 0.000000
2014-05-27, Close, 1911.91 ; Position 0.000000
2014-05-28, Close, 1909.78 ; Position 0.000000
2014-05-28, {mr}SELL CREATE, 1909.78
2014-05-29, SELL EXECUTED, 1910.60
2014-05-29, Close, 1920.03 ; Position -5.236205
2014-05-30, Close, 1923.57 ; Position -5.236205
2014-06-02, Close, 1924.97 ; Position -5.236205
2014-06-02, {mo}[Exit]BUY CREATE, 1924.

...

2015-06-12, Close, 2094.11 ; Position 4.771357
2015-06-15, Close, 2084.43 ; Position 4.771357
2015-06-15, {mo}[Exit]SELL CREATE, 2084.43
2015-06-16, SELL EXECUTED, 2084.26
2015-06-16, Close, 2096.29 ; Position 0.000000
2015-06-17, Close, 2100.44 ; Position 0.000000
2015-06-18, Close, 2121.24 ; Position 0.000000
2015-06-18, {mo}BUY CREATE, 2121.24
2015-06-19, BUY EXECUTED, 2121.06
2015-06-19, Close, 2109.99 ; Position 4.714224
2015-06-22, Close, 2122.85 ; Position 4.714224
2015-06-23, Close, 2124.20 ; Position 4.714224
2015-06-24, Close, 2108.58 ; Position 4.714224
2015-06-25, Close, 2102.31 ; Position 4.714224
2015-06-25, {mo}[Exit]SELL CREATE, 2102.31
2015-06-26, SELL EXECUTED, 2102.62
2015-06-26, Close, 2101.49 ; Position 0.000000
2015-06-29, Close, 2057.64 ; Position 0.000000
2015-06-29, {mr}BUY CREATE, 2057.64
2015-06-30, BUY EXECUTED, 2061.19
2015-06-30, Close, 2063.11 ; Position 4.859937
2015-07-01, Close, 2077.42 ; Position 4.859937
2015-07-02, Close, 2076.78 ; Position 4.859937

2016-01-06, Close, 1990.26 ; Position 4.852037
2016-01-06, {mo}[Exit]SELL CREATE, 1990.26
2016-01-07, SELL EXECUTED, 1985.32
2016-01-07, Close, 1943.09 ; Position 0.000000
2016-01-07, {mr}BUY CREATE, 1943.09
2016-01-08, BUY EXECUTED, 1945.97
2016-01-08, Close, 1922.03 ; Position 5.146442
2016-01-11, Close, 1923.67 ; Position 5.146442
2016-01-12, Close, 1938.68 ; Position 5.146442
2016-01-13, Close, 1890.28 ; Position 5.146442
2016-01-13, {mo}[Exit]SELL CREATE, 1890.28
2016-01-14, SELL EXECUTED, 1891.68
2016-01-14, Close, 1921.84 ; Position 0.000000
2016-01-15, Close, 1880.33 ; Position 0.000000
2016-01-15, {mr}BUY CREATE, 1880.33
2016-01-19, BUY EXECUTED, 1888.66
2016-01-19, Close, 1881.33 ; Position 5.318215
2016-01-20, Close, 1859.33 ; Position 5.318215
2016-01-21, Close, 1868.99 ; Position 5.318215
2016-01-22, Close, 1906.90 ; Position 5.318215
2016-01-25, Close, 1877.08 ; Position 5.318215
2016-01-26, Close, 1903.63 ; Position 5.318215
2016-01-27, Close, 1882.95 ; Position 5.318215

2016-07-29, {mo}SELL CREATE, 2173.60
2016-08-01, SELL EXECUTED, 2173.15
2016-08-01, Close, 2170.84 ; Position -4.600662
2016-08-02, Close, 2157.03 ; Position -4.600662
2016-08-03, Close, 2163.79 ; Position -4.600662
2016-08-04, Close, 2164.25 ; Position -4.600662
2016-08-05, Close, 2182.87 ; Position -4.600662
2016-08-05, {mo}[Exit]BUY CREATE, 2182.87
2016-08-08, BUY EXECUTED, 2183.76
2016-08-08, Close, 2180.89 ; Position 0.000000
2016-08-09, Close, 2181.74 ; Position 0.000000
2016-08-09, {mr}SELL CREATE, 2181.74
2016-08-10, SELL EXECUTED, 2182.81
2016-08-10, Close, 2175.49 ; Position -4.583498
2016-08-11, Close, 2185.79 ; Position -4.583498
2016-08-12, Close, 2184.05 ; Position -4.583498
2016-08-15, Close, 2190.15 ; Position -4.583498
2016-08-16, Close, 2178.15 ; Position -4.583498
2016-08-17, Close, 2182.22 ; Position -4.583498
2016-08-18, Close, 2187.02 ; Position -4.583498
2016-08-19, Close, 2183.87 ; Position -4.583498
2016-08-22, Close, 2182.64 ; Position -4.583498
2016-08-23, Cl

2017-02-24, Close, 2367.34 ; Position -4.277928
2017-02-27, Close, 2369.75 ; Position -4.277928
2017-02-28, Close, 2363.64 ; Position -4.277928
2017-03-01, Close, 2395.96 ; Position -4.277928
2017-03-02, Close, 2381.92 ; Position -4.277928
2017-03-03, Close, 2383.12 ; Position -4.277928
2017-03-06, Close, 2375.31 ; Position -4.277928
2017-03-07, Close, 2368.39 ; Position -4.277928
2017-03-08, Close, 2362.98 ; Position -4.277928
2017-03-08, {mr}[Exit]BUY CREATE, 2362.98
2017-03-09, BUY EXECUTED, 2363.49
2017-03-09, Close, 2364.87 ; Position 0.000000
2017-03-10, Close, 2372.60 ; Position 0.000000
2017-03-13, Close, 2373.47 ; Position 0.000000
2017-03-14, Close, 2365.45 ; Position 0.000000
2017-03-15, Close, 2385.26 ; Position 0.000000
2017-03-15, {mr}SELL CREATE, 2385.26
2017-03-16, SELL EXECUTED, 2387.71
2017-03-16, Close, 2381.38 ; Position -4.192415
2017-03-17, Close, 2378.25 ; Position -4.192415
2017-03-20, Close, 2373.47 ; Position -4.192415
2017-03-20, {mr}[Exit]BUY CREATE, 2373.47

2017-09-12, Close, 2496.48 ; Position 0.000000
2017-09-13, Close, 2498.37 ; Position 0.000000
2017-09-14, Close, 2495.62 ; Position 0.000000
2017-09-15, Close, 2500.23 ; Position 0.000000
2017-09-18, Close, 2503.87 ; Position 0.000000
2017-09-19, Close, 2506.65 ; Position 0.000000
2017-09-20, Close, 2508.24 ; Position 0.000000
2017-09-21, Close, 2500.60 ; Position 0.000000
2017-09-22, Close, 2502.22 ; Position 0.000000
2017-09-25, Close, 2496.66 ; Position 0.000000
2017-09-26, Close, 2496.84 ; Position 0.000000
2017-09-27, Close, 2507.04 ; Position 0.000000
2017-09-28, Close, 2510.06 ; Position 0.000000
2017-09-29, Close, 2519.36 ; Position 0.000000
2017-10-02, Close, 2529.12 ; Position 0.000000
2017-10-02, {mr}SELL CREATE, 2529.12
2017-10-03, SELL EXECUTED, 2530.34
2017-10-03, Close, 2534.58 ; Position -3.953944
2017-10-04, Close, 2537.74 ; Position -3.953944
2017-10-05, Close, 2552.07 ; Position -3.953944
2017-10-06, Close, 2549.33 ; Position -3.953944
2017-10-09, Close, 2544.73 ; Po